In [1]:
# Ensure required packages are installed in this Python environment
import sys
import subprocess
required = ['pandas', 'matplotlib', 'seaborn']
for pkg in required:
    try:
        __import__(pkg)
    except Exception:
        print(f'Installing {pkg}...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])
print('Required packages are installed (or were installed just now).')

Installing matplotlib...
Installing seaborn...
Required packages are installed (or were installed just now).


In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv("preprocessedPhase3FoodFacts.csv")

In [6]:
tabular_cols = df.select_dtypes(include=["number"]).columns
print(tabular_cols)

Index(['barcode', 'nutriscore_letter', 'nova_group', 'energy_kcal_100g',
       'fat_100g', 'saturated_fat_100g', 'carbohydrates_100g', 'sugars_100g',
       'fiber_100g', 'proteins_100g', 'salt_100g', 'contains_palm_oil',
       'vegetarian_status', 'vegan_status', 'nutrient_level_fat',
       'nutrient_level_saturated_fat', 'nutrient_level_sugars',
       'nutrient_level_salt', 'ecoscore_grade', 'ecoscore_score',
       'carbon_footprint_100g', 'additives_count', 'sugar_ratio',
       'energy_density', 'protein_ratio', 'macro_balance', 'healthy_score',
       'log_energy_kcal_100g', 'log_fat_100g', 'log_sugars_100g',
       'log_salt_100g', 'has_image', 'has_image_160'],
      dtype='object')


In [7]:
feature_cols = [
    'nova_group', 'energy_kcal_100g', 'fat_100g',
    'saturated_fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g',
    'proteins_100g', 'salt_100g', 'contains_palm_oil', 'vegetarian_status', 'vegan_status',
    'nutrient_level_fat', 'nutrient_level_saturated_fat',
    'nutrient_level_sugars', 'nutrient_level_salt', 'ecoscore_grade', 'ecoscore_score',
    'carbon_footprint_100g', 'additives_count', 'sugar_ratio',
    'energy_density', 'protein_ratio', 'macro_balance', 'healthy_score',
    'log_energy_kcal_100g', 'log_fat_100g', 'log_sugars_100g',
    'log_salt_100g'
]

In [8]:
#Show ALL columns permanently in this notebook
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_rows", None)

In [ ]:
# Make sure nova_group is numeric if it isn't already
df["nova_group"] = pd.to_numeric(df["nova_group"], errors="coerce")
df["vegetarian_status"] = pd.to_numeric(df["vegetarian_status"], errors="coerce")
df["vegan_status"] = pd.to_numeric(df["vegan_status"], errors="coerce")

# Class balance of Nutri-Score
print(df["nutriscore_letter"].value_counts(dropna=False))
print("\nClass proportions:")
print(df["nutriscore_letter"].value_counts(normalize=True))

# Basic stats for your features
print("\n=== Numeric feature summary ===")
print(df[feature_cols].describe(percentiles=[0.25, 0.5, 0.75]))

# Missingness per feature
print("\n=== Missing value ratios ===")
print(df[feature_cols].isna().mean().sort_values(ascending=False))

nutriscore_letter
5.0    1157
1.0     877
4.0     705
3.0     346
2.0     100
Name: count, dtype: int64

Class proportions:
nutriscore_letter
5.0    0.363265
1.0    0.275353
4.0    0.221350
3.0    0.108634
2.0    0.031397
Name: proportion, dtype: float64

=== Numeric feature summary ===
        nova_group  energy_kcal_100g     fat_100g  saturated_fat_100g  \
count  3185.000000       3185.000000  3185.000000         3185.000000   
mean      3.762637        458.164521    24.332963            7.313153   
std       0.425533        127.161979    16.657767            7.766243   
min       3.000000          0.000000     0.000000            0.000000   
25%       4.000000        393.000000    12.000000            1.600000   
50%       4.000000        475.000000    22.300000            4.400000   
75%       4.000000        539.000000    33.800000           11.300000   
max       4.000000        786.000000    75.000000           46.670000   

       carbohydrates_100g  sugars_100g   fiber_100g  p

In [10]:
# Group stats by Nutri-Score
group_stats = df.groupby("nutriscore_letter")[feature_cols].mean()
print("\n=== Mean values by Nutri-Score ===")
print(group_stats)


=== Mean values by Nutri-Score ===
                   nova_group  energy_kcal_100g   fat_100g  \
nutriscore_letter                                            
1.0                  3.679590        430.566705  27.329935   
2.0                  3.620000        431.000000  24.824170   
3.0                  3.601156        430.731214  19.565720   
4.0                  3.775887        456.201418  21.537027   
5.0                  3.878133        490.831461  25.148119   

                   saturated_fat_100g  carbohydrates_100g  sugars_100g  \
nutriscore_letter                                                        
1.0                          4.028640           34.021664     2.996178   
2.0                          3.524270           46.767370     3.424300   
3.0                          2.500789           53.695477     8.690142   
4.0                          5.479489           56.190532    13.845850   
5.0                         12.686719           57.859230    33.767151   

          